In [5]:
import sqlite3
import pandas as pd
from contextlib import closing

In [18]:
class SqliteCore(object):
    def __init__(self, db_path):
        self.connection = sqlite3.connect(db_path)
        
    def creat_table(self, table_name, para_list):
        sql_cmd = 'CREATE TABLE IF NOT EXISTS {}('.format(table_name)
        for idx in range(len(para_list) - 1):
            para_list[idx] += ','
        for para in para_list:
            sql_cmd += para
        sql_cmd += ')'
        self.connection.execute(sql_cmd)
    
    def _add_data(self, table_name, data_df, exists_type='append'):
        instrument_data.to_sql(table_name, con=self.connection, if_exists=exists_type, index=False)

In [19]:
sql_core = SqliteCore('test.db')
sql_core.creat_table('instrument', [
    'instrument_id int primary key not null',
    'description text not null',
    'ticker text not null',
    'maturity datetime',
    'strike real',
    'option_type text'
])

instrument_data = pd.DataFrame({
    'instrument_id': 2,
    'description': 'AAPL_stock',
    'ticker': 'AAPL',
}, index=[0])
sql_core._add_data('instrument', instrument_data)

CREATE TABLE IF NOT EXISTS instrument(instrument_id int primary key not null,description text not null,ticker text not null,maturity datetime,strike real,option_type text)


IntegrityError: UNIQUE constraint failed: instrument.instrument_id

In [20]:
pd.read_sql('select * from instrument', sql_core.connection)

,instrument_id,description,ticker,maturity,strike,option_type
0,1,AAPL_stock,AAPL,None,None,None
1,2,AAPL_stock,AAPL,None,None,None


In [21]:
sql_core.creat_table('market', [
    'market_id int primary key not null',
    'instrument_id',
    'timestamp datetime not null',
    'bid real',
    'ask real',
    'mid real',
    'delta real',
    'IV real',
    'gamma real',
    'thela real',
    'vega real',
    'foreign key (instrument_id) references instrument (instrument_id)'
])

CREATE TABLE IF NOT EXISTS market(market_id int primary key not null,instrument_id,timestamp datetime not null,bid real,ask real,mid real,delta real,IV real,gamma real,thela real,vega real,foreign key (instrument_id) references instrument (instrument_id))


In [23]:
# order是关键字
sql_core.creat_table('my_order', [
    'order_id int primary key not null',
    'instrument_id',
    'timestamp datetime not null',
    'qty real not null',
    'order_type real not null',
    'order_price real not null',
    'foreign key (instrument_id) references instrument (instrument_id)'
])

CREATE TABLE IF NOT EXISTS my_order(order_id int primary key not null,instrument_id,timestamp datetime not null,qty real not null,order_type real not null,order_price real not null,foreign key (instrument_id) references instrument (instrument_id))


In [24]:
sql_core.creat_table('trade', [
    'trade_id int primary key not null',
    'instrument_id',
    'timestamp datetime not null',
    'qty real not null',
    'price real not null',
    'reason text',
    'foreign key (instrument_id) references instrument (instrument_id)'
])

CREATE TABLE IF NOT EXISTS trade(trade_id int primary key not null,instrument_id,timestamp datetime not null,qty real not null,price real not null,reason text,foreign key (instrument_id) references instrument (instrument_id))


In [25]:
sql_core.creat_table('position', [
    'position_id int primary key not null',
    'instrument_id',
    'replace_position_id',
    'startdate datetime not null',
    'enddate datetime not null',
    'foreign key (instrument_id) references instrument (instrument_id)',
    'foreign key (replace_position_id) references position (replace_position_id)'
])

CREATE TABLE IF NOT EXISTS position(position_id int primary key not null,instrument_id,replace_position_id,startdate datetime not null,enddate datetime not null,foreign key (instrument_id) references instrument (instrument_id),foreign key (replace_position_id) references position (replace_position_id))
